In [ ]:
from scripts import support, metrics, vectorization as v, aggregation
from scripts.kuroda import kuroda
from scripts.insd import insd
from scripts.nras import nras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
%matplotlib inline

## Prepaire data

In [ ]:
#CODES


df = pd.read_excel("./data/Use_SUT_Framework_2007_2012_DET.xlsx", "NAICS Codes", skiprows=4)
df = df.iloc[:-6,:-1]
df = df.dropna(axis=0, how = 'all')
df = df.fillna(0)

codes = np.array(df)

In [ ]:
#DET 2007


df = pd.read_excel("./data/Use_SUT_Framework_2007_2012_DET.xlsx", "2007")
df = df.iloc[5:-11,2:-21]
df = df.fillna(0)

DET_2007 = np.array(df)
DET_2007 = np.where(DET_2007=='...', 0, DET_2007)
dt_2007 = v.tovector(DET_2007).astype(float)
# np.shape(DET_2007)

In [ ]:
#DET 2012


df = pd.read_excel("./data/Use_SUT_Framework_2007_2012_DET.xlsx", "2012")
df = df.iloc[5:-11,2:-21]
df = df.fillna(0)

DET_2012 = np.array(df)
DET_2012 = np.where(DET_2012=='...', 0, DET_2012)
dt_2012 = v.tovector(DET_2012).astype(float)


#Зануляем элементы, в которых меняется знак
det_2007 = dt_2007
det_2012 = dt_2012
det_2007[dt_2007 * dt_2012 < 0] = 0
det_2012[dt_2007 * dt_2012 < 0] = 0
DET_2012[DET_2007 * DET_2012 < 0] = 0


#Считаем итоги по строкам и столбцам
rows_sums_2012 = np.sum(DET_2012, axis=1)
columns_sums_2012 = np.sum(DET_2012, axis=0)
totals_2012 = np.concatenate((rows_sums_2012, columns_sums_2012)).astype(float)[np.newaxis].T
# DET_2012

In [ ]:
#SUM 2007


df = pd.read_excel("./data/Use_SUT_Framework_1997-2017_SUM.xlsx", "2007")
df = df.iloc[6:-9,2:-21]
df = df.fillna(0)

SUM_2007 = np.array(df)
SUM_2007 = np.where(SUM_2007=='...', 0, SUM_2007)

SUM_2007[[72, 71]] = SUM_2007[[71, 72]]
sum_2007 = v.tovector(SUM_2007).astype(float)

In [ ]:
#SUM 2012


df = pd.read_excel("./data/Use_SUT_Framework_1997-2017_SUM.xlsx", "2012")
df = df.iloc[6:-9,2:-21]
df = df.fillna(0)

SUM_2012 = np.array(df)
SUM_2012 = np.where(SUM_2012=='...', 0, SUM_2012)

SUM_2012[[72, 71]] = SUM_2012[[71, 72]]
sum_2012 = v.tovector(SUM_2012).astype(float)
# SUM_2012

In [ ]:
#SECT 2007


df = pd.read_excel("./data/Use_SUT_Framework_1997-2017_SECT.xlsx", "2007")
df = df.iloc[6:-9, 2:-7]
df = df.fillna(0)

SECT_2007 = np.array(df)
SECT_2007 = np.where(SECT_2007=='...', 0, SECT_2007)
sect_2007 = v.tovector(SECT_2007).astype(float)
# SECT_2007

In [ ]:
#SECT 2012


df = pd.read_excel("./data/Use_SUT_Framework_1997-2017_SECT.xlsx", "2012")
df = df.iloc[6:-9, 2:-7]
df = df.fillna(0)

SECT_2012 = np.array(df)
SECT_2012 = np.where(SECT_2012=='...', 0, SECT_2012)
sect_2012 = v.tovector(SECT_2012).astype(float)
# SECT_2012

In [ ]:
#parsing aggregation codes


sect_rows, sect_columns, sum_rows, sum_columns = aggregation.parse_agg(codes)

In [ ]:
#Составляем матрицы-ключи агрегации и векторизованный вид
#Для аггрегации в SUM (73,71)

left_sum = aggregation.make_keym(405, 73, sum_rows)
right_sum = aggregation.make_keym(405, 71, sum_columns)

In [ ]:
#Составляем матрицы-ключи агрегации и векторизованный вид
#Для аггрегации в SECT (17,15)

left_sect = aggregation.make_keym(405, 17, sect_rows)
right_sect = aggregation.make_keym(405, 15, sect_columns)

In [ ]:
g_to_sum = aggregation.keys_to_g(left_sum, right_sum)
g_to_sect = aggregation.keys_to_g(left_sect, right_sect)

tsum_2012 = g_to_sum @ det_2012
tsect_2012 = g_to_sect @ det_2012


In [ ]:
# Add totals to aggregation
sum_2012_with_totals = np.vstack((tsum_2012, totals_2012))
sect_2012_with_totals = np.vstack((tsect_2012, totals_2012))

In [ ]:
from scripts import support, metrics, vectorization as v, aggregation
def experiment(tables_kind, method_name, bench_matrix, origin_matrix, target_matrix, experiment_type, sparsed = True, save = True, totals = False):
    """
    Получение результата эксперимента

    Parameters
    ----------
    tables_kind: string
        степень аггрегации таблиц
    method_name: string
        метод для получения новой матрицы
    bench_matrix: np.array или sparse матрица
        бенчмарк таблица
    origin_matrix: np.array или sparse матрица
        оригинальная таблица
    target_matrix: np.array или sparse матрица
        целевая таблица
    experiment_type: string
        агрегация\дезаггрегация\проекция
    sparse: bool
        использовать sparse матрицу или np.array
    Returns
    -------
    a: float
        значение метрики
    """
    if experiment_type == 'дезаггрегация':
        if tables_kind == 'sum':
            g = aggregation.keys_to_g(left_sum, right_sum)
        elif tables_kind == 'sect':
            g = aggregation.keys_to_g(left_sect, right_sect)
        else:
            raise Exception("Wrong tables_kind", tables_kind)
    elif experiment_type == 'проекция':
        ttls = v.addrows((405,405))
        g = v.addcolumns((405,405), ttls)
    else:
        raise Exception("Wrong experiment_type",experiment_type)  
    

    if sparsed:
        g = csr_matrix(g)
        
    if totals:
        g = v.addrows((405,405), g, sparsed=sparsed)
        g = v.addcolumns((405,405), g, sparsed=sparsed)
    
    # For Kuroda
    if len(method_name.split('_')) > 1:
        
        method, mtype = method_name.split('_')
        method = eval(method)
        results = np.round(method(g, eval(bench_matrix), eval(origin_matrix), mtype=int(mtype), sparsed=sparsed))
    else:
        method = eval(method_name)
        results = method(g, eval(bench_matrix), eval(origin_matrix), sparsed=sparsed)
    metrics_values = metrics.get_values(np.round(results), eval(target_matrix), save=True, shape = (405,405))
    
    # Save metrics' results
    if save:
        info = {'method':method_name, 'experiment_type':experiment_type, 'target_matrix':target_matrix, 
                'origin_matrix':origin_matrix, 'bench_matrix':bench_matrix}
        info = {**info,**metrics_values}

        res_table = pd.read_excel('results/results_us_use.xlsx')
        res_table = res_table.append([info])
        res_table.to_excel('results/results_us_use.xlsx', index=None,encoding='utf-8')

    return metrics_values 

def experiments(tables_kind, bench_matrix, origin_matrix, target_matrix, experiment_type, totals = False):
    methods_list = ['nras','insd','kuroda_3','kuroda_2','kuroda_1']
    sparsed = True
    for i, method in enumerate(methods_list):
        print('Method:', method)
        if i > 2:
            sparsed = False
        experiment(tables_kind, method, bench_matrix, origin_matrix, target_matrix, experiment_type, 
                   sparsed = sparsed, totals = totals)
            

In [ ]:
#Эксперимент: дезаггрегация таблицы DET_2012 из таблицы SUM_2012
#Бенчмарк: DET_2007
#Ограничения: таблица SUM_2012
#Методы: NRAS, точность 0.01, 100 иттераций, INSD, Kuroda(1,2,3)

experiments('sum', 'det_2007', 'sum_2012', 'det_2012', 'дезаггрегация')

In [ ]:
#Эксперимент: дезаггрегация таблицы DET_2012 из таблицы SECT_2012
#Бенчмарк: DET_2007
#Ограничения: таблица SECT_2012
#Методы: NRAS, точность 0.01, 100 иттераций, INSD, Kuroda(1,2,3)

experiments('sect', 'det_2007', 'sect_2012', 'det_2012', 'дезаггрегация')

In [ ]:
#Эксперимент: проекция таблицы DET_2012 из окаймляющих итогов
#Бенчмарк: DET_2007
#Ограничения: окаймляющие итоги
#Методы: NRAS, точность 0.01, 100 иттераций, INSD, Kuroda(1,2,3)

experiments('', 'det_2007', 'totals_2012', 'det_2012', 'проекция')

In [ ]:
#Эксперимент: проекция+дезагрегация таблицы DET_2012 из окаймляющих итогов и таблицы SUM_2012
#Бенчмарк: DET_2007
#Ограничения: окаймляющие итоги и таблица SUM_2012
#Методы: NRAS, точность 0.01, 100 иттераций, INSD, Kuroda(1,2,3)

experiments('sum', 'det_2007', 'sum_2012_with_totals' , 'det_2012', 'дезаггрегация', totals = True)

In [ ]:
#Эксперимент: проекция+дезагрегация таблицы DET_2012 из окаймляющих итогов и таблицы SECT_2012
#Бенчмарк: DET_2007
#Ограничения: окаймляющие итоги и таблица SECT_2012
#Методы: NRAS, точность 0.01, 100 иттераций, INSD, Kuroda(1,2,3)

experiments('sect', 'det_2007', 'sect_2012_with_totals' , 'det_2012', 'дезаггрегация', totals = True)